# Open RiOMar HEALPix_Zarr data  

## Prerequisites
1. . Install:
```bash
pip install xdggs
```


## What this notebook does
- Loads the Healpix_ Zarr riomar data 
- visualise with xdggs


## Steps
1.	**Setup & load inputs**

Load the  Zarr file 

2.	**visualise **



## Output dataset conventions
- meta data shoudl be updated to aligne fair practice? 



## Step 1 — Setup & load zarr file 

In [ ]:
import xarray as xr
import xdggs
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
zarr_hp_file_path = 'https://data-fair2adapt.ifremer.fr/riomar-zarr_tina/small_hp.zarr'
zarr_hp_file_path = '/scale/riomar-zarr_tina/small_hp.zarr'
zarr_hp_file_path = 'https://data-fair2adapt.ifremer.fr/riomar-zarr_tina/test3.zarr'
zarr_hp_file_path = './riomar-zarr_tina/test3.zarr'

ds = xr.open_zarr(zarr_hp_file_path)
#ds = ds.persist().sortby("time_counter")
ds

## Step 2 -- resample the data and visualise


### 2.1 plot daily mean 

In [ ]:
# Make time monotonic (required for resample)

# Daily mean
ds_mean_daily = ds.resample(time_counter="1D").mean(keep_attrs=True)
ds_mean_daily

In [ ]:
(
    ds_mean_daily.compute()
    .pipe(xdggs.decode)
    .dggs.assign_latlon_coords()
    .dggs.explore(alpha=0.3)
)

### 2.2 plot time series of one point   

chose one point and make time series plot


In [ ]:
import numpy as np

# 1) Midpoint of your bbox
lon_min, lon_max = -2.8, -1.97333
lat_min, lat_max = 47.04367, 47.31558

lon_mid = (lon_min + lon_max) / 2.0
lat_mid = (lat_min + lat_max) / 2.0

print("midpoint:", lon_mid, lat_mid)

# 2) Pick up the level from the dataset's cell_ids attribute
level = ds.cell_ids.attrs['level']

# Common attribute keys people use — adjust if your file uses another name


print("HEALPix level:", level)

# 3) Compute the Helix cell_id for that lon/lat using healpix_geo
#    (function names vary a bit between packages/versions; try the usual one)
import healpix_geo as hpg

cell_id = hpg.nested.lonlat_to_healpix(lon_mid, lat_mid, depth=level,ellipsoid= "WGS84")

print("HEALPix cell_id:", cell_id)

# 4) Select that cell from ds
# If cell_ids is a coordinate along dimension "cell_ids", this should work:
ds_point=ds.sel(cell_ids=cell_id).pipe(xdggs.decode).dggs.assign_latlon_coords().compute()


In [ ]:
import matplotlib.pyplot as plt

ds_point.zeta.plot()


In [ ]:

plt.savefig("zeta.png", dpi=300, bbox_inches="tight")
plt.close()